# Set up

In [1]:
!git clone https://github.com/aliswh/lastfm
!cd lastfm; pip install -r requirements.txt

Cloning into 'lastfm'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 228 (delta 125), reused 163 (delta 68), pack-reused 0
Receiving objects: 100% (228/228), 39.75 KiB | 3.97 MiB/s, done.
Resolving deltas: 100% (125/125), done.
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 41.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f230d5e810df0ebafaadb1666ad32b048fdd3800e73161a890f4c032113c5073
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from py4j.java_gateway import UserHelpAutoCompletion
import pyspark
from pyspark.sql import  SparkSession
from lastfm.src.ingestion_layer.googlestorage import *
from lastfm.src.ingestion_layer.pylastsource import *
from lastfm.src.ingestion_layer.config import *
from lastfm.src.ingestion_layer.batchwriter import *
import json
import datetime

# PySpark Context

In [3]:
sc = pyspark.SparkContext("local[*]")
spark = SparkSession(sc)

storage = GoogleStorageJSON('./lastfm/src/ingestion_layer/creds.json',bucket_name)
source = PyLastSource(AUTH_DATA)

Batch job

In [8]:
#writer = BatchWriter(source, storage)
#writer.write('subfolder', 'dars4', 5, 5, debug=True)
#storage.delete('subfolder/artists')

[User] {'user': 'c4zt96', 'playcount': 11580, 'reg_date': '1627094549', 'country': 'Brazil', 'url': 'https://www.last.fm/user/c4zt96', 'id': '0d84bc5dedacad885f9fa1df8a0b8d8ca8467c80'}
[Artist] {'artist': 'Queens of the Stone Age', 'bio': 'Queens of the Stone Age is a rock band from Palm Desert, California, United States. The band is frequently labeled stoner rock, although they reject the label. They developed a style of riff-oriented, heavy music which the band\'s founder and mastermind Josh Homme described as "robot rock", saying that he "wanted to create a heavy sound based on a solid jam, just pound it into your head". Since then, their sound has evolved to incorporate a variety of different styles and influences. <a href="https://www.last.fm/music/Queens+of+the+Stone+Age">Read more on Last.fm</a>', 'url': 'https://www.last.fm/music/queens%2bof%2bthe%2bstone%2bage', 'top_tracks': [{'artist': 'Queens of the Stone Age', 'title': 'No One Knows', 'album': 'Songs for the Deaf', 'durati

Missing track

In [8]:
source.get('track', 'Rich Nanni', 'Macaroni')

{'album': None,
 'artist': 'Rich Nanni',
 'duration': 0,
 'id': 'cb5617ef265b3fe5e48c859d7016a09dcf4a4215',
 'tags': [],
 'title': 'Macaroni',
 'url': 'https://www.last.fm/music/rich%2bnanni/_/macaroni'}

# Read from storage

In [10]:
recent_tracks = storage.read('recent_tracks',dir=True)
for recent_track in recent_tracks:
  print(recent_track)
recent_tracks_rdd = sc.parallelize(recent_tracks)

['amarxl', {'artist': 'Kim Petras', 'title': 'XXX', 'date': '17 Feb 2022, 14:51', 'album': 'Slut Pop', 'timestamp': '1645109503', 'id': '4918d56108433a5a44194077398663d691a68741'}, {'artist': 'Kim Petras', 'title': 'Treat Me Like a Slut', 'date': '17 Feb 2022, 14:49', 'album': 'Slut Pop', 'timestamp': '1645109385', 'id': '9e41f80918c4172b2b70bbe2bbc3b1042af80290'}, {'artist': 'Kim Petras', 'title': 'Slut Pop', 'date': '17 Feb 2022, 14:47', 'album': 'Slut Pop', 'timestamp': '1645109268', 'id': 'cab054b76d7247c17c2475a09899211dd7bb9090'}, {'artist': 'Kim Petras', 'title': 'Your Wish Is My Command', 'date': '17 Feb 2022, 14:44', 'album': 'Slut Pop', 'timestamp': '1645109093', 'id': 'ead01e99eb5412fd45cc8732fffbac46559c125a'}, {'artist': 'Kim Petras', 'title': 'They Wanna Fuck', 'date': '17 Feb 2022, 14:42', 'album': 'Slut Pop', 'timestamp': '1645108943', 'id': '0edae09d0b08e454cc9c181d6debf6a994a1cee8'}, {'artist': 'Kim Petras', 'title': 'Throat Goat', 'date': '17 Feb 2022, 14:40', 'album

In [11]:
tracks = storage.read('tracks',dir=True)
tracks_rdd = sc.parallelize(tracks)
tracks_rdd.take(1)

[{'album': 'Me Rehúso',
  'artist': 'Danny Ocean',
  'duration': 206000,
  'id': '02ea699815e47f005f5fdccf507d68ac123f85bd',
  'tags': ['pop', 'latin', 'summer', 'party', 'latin pop'],
  'title': 'Me Rehúso',
  'url': 'https://www.last.fm/music/danny%2bocean/_/me%2breh%25c3%25baso'}]

In [12]:
tracks_rdd.count()

174

In [13]:
artists = storage.read('artists',dir=True)
artists_rdd = sc.parallelize(artists)
artists_rdd.take(1)

[{'artist': 'Kim Petras',
  'bio': 'Kim Petras (born August 27, 1992) is a German singer and songwriter. Between 2016 and 2020, she released music as an independent artist under her own imprint, BunHead Records, before signing with Republic Records.\n\nPetras began recording music as a teenager, releasing her debut EP One Piece of Tape in 2011. She independently released her debut single in 2017, "I Don\'t Want It at All", which went on to top several viral music charts on Spotify. The song was followed by Billboard-charting singles "Feeling of Falling" (with Cheat Codes) <a href="https://www.last.fm/music/Kim+Petras">Read more on Last.fm</a>',
  'id': '015b6e8359bdf7f09ef4c3aac6e97e7d43553aea',
  'top_tracks': [{'album': 'ERA 1',
    'artist': 'Kim Petras',
    'duration': 227000,
    'id': '47df2ab45a07ee23217fb25b69bdf7a282cabac3',
    'tags': ['pop', 'synthpop', 'seen live', 'electropop', 'dance-pop'],
    'title': 'Heart to Break',
    'url': 'https://www.last.fm/music/kim%2bpetra

# User features

1. Extract listening sessions from users:
  * list of concatenated songs with a play events +-10 sec after the actual song completed
2. Know statistics about these sessions:
  * average number of tracks
  * average session per user
  * session lengths

## Extract listening sessions

In [276]:
def to_datetime(x):
  timestamp_date = lambda ts: datetime.datetime.fromtimestamp(int(ts))
  date_date = lambda ts: datetime.datetime.strptime(ts, '%d %b %Y, %H:%M')
  x[1]['date'] = date_date(x[1]['date'])
  x[1]['timestamp'] = timestamp_date(x[1]['timestamp'])
  return x

sessions = recent_tracks_rdd.flatMap(lambda x: [(x[0],y) for y in x[1:]]) \
                            .map(lambda x: to_datetime(x))
sessions.take(1)  

[('amarxl',
  {'album': 'Slut Pop',
   'artist': 'Kim Petras',
   'date': datetime.datetime(2022, 2, 17, 14, 51),
   'id': '4918d56108433a5a44194077398663d691a68741',
   'timestamp': datetime.datetime(2022, 2, 17, 14, 51, 43),
   'title': 'XXX'})]

In [277]:
def timestamp_to_delta(ts1, ts2):
  return int((ts2 - ts1).total_seconds() * 1000)


sessions = sessions.join(sessions) \
                   .filter(lambda x: x[1][0]['date'] < x[1][1]['date']) \
                   .map(lambda x: ((x[0], x[1][0]['title'], x[1][0]['artist'], x[1][0]['date']), timestamp_to_delta(x[1][0]['timestamp'], x[1][1]['timestamp']))) \
                   .reduceByKey(lambda x,y: min(x,y))

sessions.take(1)  

[(('bellagecs',
   'Fat Funny Friend',
   'Maddie Zahm',
   datetime.datetime(2022, 2, 17, 4, 21)),
  215000)]

In [278]:
sessions = sessions.map(lambda x: ((x[0][1], x[0][2]), (x[0][0], x[0][3], x[1])))
sessions.take(100)

[(('Fat Funny Friend', 'Maddie Zahm'),
  ('bellagecs', datetime.datetime(2022, 2, 17, 4, 21), 215000)),
 (('this is how i learn to say no', 'Emeline'),
  ('bellagecs', datetime.datetime(2022, 2, 17, 4, 13), 217000)),
 (('Attracted to You', 'pinkpantheress'),
  ('bellagecs', datetime.datetime(2022, 2, 17, 4, 11), 124000)),
 (('Complicated', "Olivia O'Brien"),
  ('bellagecs', datetime.datetime(2022, 2, 17, 1, 16), 10462000)),
 (('Freakum Dress', 'Beyoncé'),
  ('bellagecs', datetime.datetime(2022, 2, 17, 1, 7), 416000)),
 (('Reckless', 'Madison Beer'),
  ('bellagecs', datetime.datetime(2022, 2, 16, 4, 59), 200000)),
 (('LOVERBOY', 'Killboy'),
  ('bellagecs', datetime.datetime(2022, 2, 16, 4, 44), 158000)),
 (('Prisoner (feat. Dua Lipa)', 'Miley Cyrus'),
  ('bellagecs', datetime.datetime(2022, 2, 15, 22, 56), 386000)),
 (('Late To The Party', 'Emei'),
  ('bellagecs', datetime.datetime(2022, 2, 15, 22, 51), 321000)),
 (('Treat Me Like a Slut', 'Kim Petras'),
  ('bellagecs', datetime.datetim

In [279]:
tr_rdd = tracks_rdd.map(lambda x: ((x['title'],x['artist']),x['duration']))
tr_rdd.take(1)

[(('Me Rehúso', 'Danny Ocean'), 206000)]

In [280]:
sessions_tracks = sessions.join(tr_rdd)\
                          .mapValues(lambda x: (x[0][0], x[0][1], x[1], x[0][2]))
sessions_tracks.take(10) # sx from listening session, dx from track

[(('Reason', 'PinkPantheress'),
  ('bellagecs', datetime.datetime(2022, 2, 15, 2, 45), 0, 260000)),
 (('Murder Your Memory', 'Title Fight'),
  ('c4zt96', datetime.datetime(2022, 2, 17, 3, 52), 156000, 173000)),
 (('Booty Call', 'Brokencyde'),
  ('c4zt96', datetime.datetime(2022, 2, 16, 5, 29), 204000, 110000)),
 (('XXX', 'Kim Petras'),
  ('amarxl', datetime.datetime(2022, 2, 17, 14, 35), 0, 124000)),
 (('XXX', 'Kim Petras'),
  ('bellagecs', datetime.datetime(2022, 2, 17, 4, 19), 0, 139000)),
 (('XXX', 'Kim Petras'),
  ('bellagecs', datetime.datetime(2022, 2, 15, 3, 49), 0, 123000)),
 (('XXX', 'Kim Petras'),
  ('amarxl', datetime.datetime(2022, 2, 17, 0, 15), 0, 125000)),
 (('XXX', 'Kim Petras'),
  ('bellagecs', datetime.datetime(2022, 2, 15, 3, 58), 0, 55957000)),
 (('Sweet Tooth Stomp', 'Moshi Monsters'),
  ('Springyaurora', datetime.datetime(2022, 2, 17, 14, 15), 233000, 234000)),
 (('Sweet Tooth Stomp', 'Moshi Monsters'),
  ('Springyaurora', datetime.datetime(2022, 2, 17, 14, 25), 2

**Problem**: missing duration for some tracks

**Fix**: take max duration per artist to fill missing values

In [281]:
artist_max = sessions_tracks.map(lambda x: (x[0][1], x[1][2])) \
                            .reduceByKey(lambda x,y: max(x,y))
artist_max.take(10)

[('Kelly Rowland', 191000),
 ('Jake Bugg', 247000),
 ('Miley Cyrus', 169000),
 ('Mac Miller', 345000),
 ('ROSALÍA', 0),
 ('I DONT KNOW HOW BUT THEY FOUND ME', 215000),
 ('Sofía Reyes', 205000),
 ('Meg Myers', 284000),
 ('pinkpantheress', 0),
 ('Sia', 0)]

If still 0 after looking in the listening session, get one duration from the artist top tracks.

In [282]:
artist_max = artists_rdd.map(lambda x: (x['artist'], max([int(track['duration']) for track in x['top_tracks'] ]) )) \
                        .join(artist_max) \
                        .mapValues(lambda x: x[1] if x[1] != 0 else x[0])
artist_max.take(10)

[('ROSALÍA', 0),
 ('Natti Natasha', 197000),
 ('Brokencyde', 267000),
 ('Fivio Foreign', 235000),
 ('Kali Uchis', 180000),
 ('Candace', 98000),
 ('Mary J. Blige', 266000),
 ('Jake Bugg', 247000),
 ('Meg Myers', 284000),
 ('Pusha T', 214000)]

Now merge duration per track in `tr_rdd`.

In [283]:
tr_rdd.filter(lambda x: x[1] == 0).take(3)

[(('Throat Goat', 'Kim Petras'), 0),
 (('They Wanna Fuck', 'Kim Petras'), 0),
 (('Obsesion', 'Natti Natasha'), 0)]

In [284]:
fixed_tr_rdd = tr_rdd.map(lambda x: (x[0][1], (x[0][0], x[1]))) \
                     .join(artist_max) \
                     .map(lambda x: ((x[1][0][0],x[0]), max(x[1][0][1], x[1][1]))) 
fixed_tr_rdd.take(5)

[(('Family Affair', 'Mary J. Blige'), 266000),
 (('Be Without You - Kendu Mix', 'Mary J. Blige'), 266000),
 (('Throat Goat', 'Kim Petras'), 250000),
 (('They Wanna Fuck', 'Kim Petras'), 250000),
 (('XXX', 'Kim Petras'), 250000)]

There are still some zeros, but we can't do much about it.

Finally compute the listening sessions.

In [295]:
sessions_tracks = sessions.join(fixed_tr_rdd)\
                          .mapValues(lambda x: (x[0][0], x[0][1], x[1], x[0][2]))
sessions_tracks.take(10) # sx from listening session, dx from track

[(('Booty Call', 'Brokencyde'),
  ('c4zt96', datetime.datetime(2022, 2, 16, 5, 29), 267000, 110000)),
 (('Killer Queen - Remastered 2011', 'Queen'),
  ('Springyaurora', datetime.datetime(2022, 2, 17, 15, 34), 295000, 179000)),
 (('BICHOTA', 'Karol G'),
  ('diabIa', datetime.datetime(2022, 2, 17, 5, 11), 234000, 581000)),
 (("Don't Go", 'Isabela Merced'),
  ('diabIa', datetime.datetime(2022, 2, 17, 5, 6), 167000, 299000)),
 (("Don't Go", 'Isabela Merced'),
  ('diabIa', datetime.datetime(2022, 2, 17, 4, 37), 167000, 332000)),
 (('Seen It All', 'Jake Bugg'),
  ('c4zt96', datetime.datetime(2022, 2, 17, 2, 34), 247000, 170000)),
 (('Como Nossos Pais', 'Elis Regina'),
  ('c4zt96', datetime.datetime(2022, 2, 16, 4, 0), 261000, 446000)),
 (('Como Nossos Pais', 'Elis Regina'),
  ('c4zt96', datetime.datetime(2022, 2, 16, 3, 44), 261000, 967000)),
 (('no eres tu(soy yo)', 'Kali Uchis'),
  ('amarxl', datetime.datetime(2022, 2, 16, 20, 45), 180000, 121000)),
 (('Lucidity', 'You And What Army'),
  (

In [296]:
sessions_tracks = sessions_tracks.mapValues(lambda x: (x[0], (x[3]-x[2])//1000, x[2]//1000, x[1])) \
                                 .groupBy(lambda x: x[1][0]) \
                                 .map(lambda x : (x[0], list(x[1])))
sessions_tracks.take(1)

[('amarxl',
  [(('no eres tu(soy yo)', 'Kali Uchis'),
    ('amarxl', -59, 180, datetime.datetime(2022, 2, 16, 20, 45))),
   (('Blin Blin', 'Bad Gyal'),
    ('amarxl', 145, 0, datetime.datetime(2022, 2, 16, 20, 14))),
   (('Treat Me Like a Slut', 'Kim Petras'),
    ('amarxl', -133, 250, datetime.datetime(2022, 2, 17, 0, 13))),
   (('Treat Me Like a Slut', 'Kim Petras'),
    ('amarxl', -131, 250, datetime.datetime(2022, 2, 17, 14, 33))),
   (('Treat Me Like a Slut', 'Kim Petras'),
    ('amarxl', -132, 250, datetime.datetime(2022, 2, 17, 14, 49))),
   (('Prada', 'Arca'),
    ('amarxl', -64, 227, datetime.datetime(2022, 2, 16, 19, 50))),
   (('Slut Pop', 'Kim Petras'),
    ('amarxl', -133, 250, datetime.datetime(2022, 2, 17, 14, 47))),
   (('Slut Pop', 'Kim Petras'),
    ('amarxl', -133, 250, datetime.datetime(2022, 2, 17, 14, 31))),
   (('TKN (feat. Travis Scott)', 'ROSALÍA'),
    ('amarxl', 130, 0, datetime.datetime(2022, 2, 16, 19, 53))),
   (('Fuego (Florentino Remix)', 'florentino'),


If the song has a value `< threshold` (let's say 15 seconds), it means that it is the last song of the listening session, so from the next song we create a new listening session.

In [297]:
threshold = 15 # seconds

def get_sessions_per_user(x):
  all_tracks = sorted(x[1], key=lambda x: x[1][3])
  sessions = []
  temp = []
  for track in all_tracks:
    temp.append(track)
    if track[1][1] < threshold:
      sessions.append(temp)
      temp = []
  return sessions

user_sessions = sessions_tracks.map(lambda x: (x[0],get_sessions_per_user(x)))
user_sessions.take(2)

[('amarxl',
  [[(('Your Wish Is My Command', 'Kim Petras'),
     ('amarxl', 71074, 250, datetime.datetime(2022, 2, 15, 23, 49))),
    (('SAOKO', 'ROSALÍA'),
     ('amarxl', 136, 0, datetime.datetime(2022, 2, 16, 19, 38))),
    (('Mariposa', 'Isabella LoveStory'),
     ('amarxl', 1, 179, datetime.datetime(2022, 2, 16, 19, 40)))],
   [(('Fuego (Florentino Remix)', 'florentino'),
     ('amarxl', -47, 241, datetime.datetime(2022, 2, 16, 19, 43)))],
   [(('KLK', 'Arca'),
     ('amarxl', 1, 227, datetime.datetime(2022, 2, 16, 19, 47)))],
   [(('Prada', 'Arca'),
     ('amarxl', -64, 227, datetime.datetime(2022, 2, 16, 19, 50)))],
   [(('TKN (feat. Travis Scott)', 'ROSALÍA'),
     ('amarxl', 130, 0, datetime.datetime(2022, 2, 16, 19, 53))),
    (('Tiro', 'Arca'),
     ('amarxl', -88, 227, datetime.datetime(2022, 2, 16, 19, 55)))],
   [(('Golosa', 'Isabella LoveStory'),
     ('amarxl', -19, 179, datetime.datetime(2022, 2, 16, 19, 58)))],
   [(('Con Altura', 'ROSALÍA'),
     ('amarxl', 162, 0, d

## Statistics about listening sessions

average number of tracks

average session per user

session lengths

In [248]:
def avg_tracks(x):
  r,c = 0, 1
  for y in x:
    r += len(y)
    c += 1
  return r/c

user_sessions.map(lambda x: (x[0], avg_tracks(x[1]))) \
             .take(5)

[('amarxl', 1.7619047619047619),
 ('Springyaurora', 1.5714285714285714),
 ('diabIa', 2.1818181818181817),
 ('c4zt96', 1.28125),
 ('bellagecs', 1.75)]

In [251]:
def avg_session_len(x):
  r,c = 0, 1
  for y in x:
    r += y[0][1][2]
    c += 1
  return r/c/1000

user_sessions.map(lambda x: (x[0], avg_session_len(x[1])))\
             .take(5)

[('amarxl', 4.071428571428571),
 ('Springyaurora', 0.17142857142857143),
 ('diabIa', 0.6818181818181819),
 ('c4zt96', 3.556875),
 ('bellagecs', 0.9407142857142856)]